# Proyecto 01 - Parte 02
**Caso aplicativo**
El estudio “CineVision Studios” está complacido con lo que descubrió en el análisis exploratorio que su equipo le entregó. Sin embargo, le han surgido nuevas interrogantes. En la conformación de su equipo de Data Science, uno de los candidatos, en las entrevistas le ha comentado que existen algoritmos que permiten detectar patrones no evidentes en los datos que le proporcionó y “juntar” variables para que
sea más pequeño. Le ha pedido que indague un poco más en los datos y extraiga información interesante.

## Procesamiento

### Ingesta de datos

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import pyclustertend

file_path = "./movies.csv"
df = pd.read_csv(file_path, encoding="latin1")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   budget                     10000 non-null  int64  
 2   genres                     9947 non-null   object 
 3   homePage                   4193 non-null   object 
 4   productionCompany          9543 non-null   object 
 5   productionCompanyCountry   8720 non-null   object 
 6   productionCountry          9767 non-null   object 
 7   revenue                    10000 non-null  float64
 8   runtime                    10000 non-null  int64  
 9   video                      9514 non-null   object 
 10  director                   9926 non-null   object 
 11  actors                     9920 non-null   object 
 12  actorsPopularity           9913 non-null   object 
 13  actorsCharacter            9953 non-null   obje

### Normalización de datos

In [102]:
columns_with_pipes = [
    "genres", "productionCompany", "productionCompanyCountry", 
    "productionCountry", "actors", "actorsPopularity", "actorsCharacter"
]

for col in columns_with_pipes:
    df[col + "_count"] = df[col].astype(str).apply(lambda x: len(x.split("|")) if pd.notna(x) else 0)

df.rename(columns={
    'genres_count': 'genresAmount',
    'productionCompany_count': 'productionCoAmount',
    'productionCountry_count': 'productionCountriesAmount'
}, inplace=True)

df = df.loc[:, ~df.columns.duplicated()]

print("Columnas disponibles:", df.columns.tolist())

Columnas disponibles: ['id', 'budget', 'genres', 'homePage', 'productionCompany', 'productionCompanyCountry', 'productionCountry', 'revenue', 'runtime', 'video', 'director', 'actors', 'actorsPopularity', 'actorsCharacter', 'originalTitle', 'title', 'originalLanguage', 'popularity', 'releaseDate', 'voteAvg', 'voteCount', 'genresAmount', 'productionCoAmount', 'productionCountriesAmount', 'actorsAmount', 'castWomenAmount', 'castMenAmount', 'productionCompanyCountry_count', 'actors_count', 'actorsPopularity_count', 'actorsCharacter_count']


### Selección de variables

In [ ]:
numeric_columns = ["actorsPopularity", "castWomenAmount", "castMenAmount"]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

features = [
    'popularity', 'budget', 'revenue', 'runtime', 'voteCount', 'voteAvg',
    'genresAmount', 'productionCoAmount', 'productionCountriesAmount',
    'actorsPopularity', 'actorsAmount', 'castWomenAmount', 'castMenAmount'
]

X = df[features].dropna()

scaler_global = StandardScaler()
X_scaled = scaler_global.fit_transform(X)

print("Forma de X_scaled:", X_scaled.shape)

df_scaled = pd.DataFrame(X_scaled, columns=features)
print("Vista previa de los datos globalmente estandarizados:")
print(df_scaled.head())

df_scaled.to_csv("movies_normalized.csv", index=False)

Forma de X_scaled: (39, 13)
Vista previa de los datos globalmente estandarizados:
   popularity    budget   revenue   runtime  voteCount   voteAvg  \
0   -0.481159 -0.175515 -0.162621 -0.601988   3.434166  0.557957   
1   -0.151433 -0.175515 -0.162621 -0.079491   0.173006 -0.591978   
2    0.488561 -0.175515 -0.162621  0.599755   0.410408  0.027217   
3   -0.536668 -0.175515 -0.162621 -0.671654   1.872308 -0.238152   
4   -0.356378 -0.175515 -0.162621 -0.323323  -0.901553  0.734870   

   genresAmount  productionCoAmount  productionCountriesAmount  \
0     -0.535672           -0.495439                  -0.436051   
1      3.642567           -0.495439                  -0.436051   
2     -0.535672            1.574790                   3.343061   
3      1.553448            0.194637                  -0.436051   
4     -0.535672           -0.495439                  -0.436051   

   actorsPopularity  actorsAmount  castWomenAmount  castMenAmount  
0          2.624196      0.426401         1.

In [117]:
# Variables consideradas por análisis como relevantes agruparlas para su estudio
features_rendimiento = ['popularity', 'voteCount', 'voteAvg']
features_financiero  = ['budget', 'revenue']
features_produccion  = ['runtime', 'genresAmount', 'productionCoAmount', 'productionCountriesAmount']
features_reparto     = ['actorsPopularity', 'actorsAmount', 'castWomenAmount', 'castMenAmount']

def preparar_subconjunto(dataframe, features):
    data = dataframe[features].dropna()
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    return data_scaled, scaler, data.index

data_rendimiento, scaler_rendimiento, indices_rendimiento = preparar_subconjunto(df, features_rendimiento)
data_financiero, scaler_financiero, indices_financiero   = preparar_subconjunto(df, features_financiero)
data_produccion, scaler_produccion, indices_produccion   = preparar_subconjunto(df, features_produccion)
data_reparto, scaler_reparto, indices_reparto             = preparar_subconjunto(df, features_reparto)
